In [ ]:
# 06_calculadora_backend.ipynb
# Celda 1
# Carga inicial y unificación de capas

%run ./00_template.py
import geopandas as gpd
import pandas as pd

# 1. Rescatamos el diccionario completo del notebook 02
SERVICE_LAYERS = {
    "salud": "establecimientos_salud",
    "educacion_escolar": "establecimientos_educacion",
    "educacion_superior": "establecimientos_educacion_superior",
    "supermercados": "osm_supermercados",
    "almacenes_barrio": "osm_almacenes_barrio",
    "bancos": "osm_bancos",
    "ferias_libres": "ferias_libres",
    "areas_verdes": "areas_verdes",
    "cuarteles_carabineros": "cuarteles_carabineros",
    "companias_bomberos": "companias_bomberos",
    "estadios": "osm_estadios",
    "malls": "osm_malls",
    "bencineras": "osm_bencineras",
    "iglesias": "osm_iglesias",
    "museos": "osm_museos",
    "infraestructura_deportiva": "infraestructura_deportiva",
    "paradas_micro": "paradas_micro",
    "paradas_metro_tren": "paradas_metro_tren",
}

def cargar_servicios_unificados(gpkg_path, layers_dict):
    """
    Carga todas las capas, les pone una etiqueta y las une en un solo GeoDataFrame.
    """
    lista_gdfs = []
    
    print("Iniciando carga masiva de servicios...")
    for category, layer_name in layers_dict.items():
        try:
            # Cargamos la capa individual
            gdf = gpd.read_file(gpkg_path, layer=layer_name)
            
            # Nos aseguramos de mantener solo la geometría y agregar la categoría
            gdf = gdf[['geometry']].copy()
            gdf['tipo_servicio'] = category
            
            lista_gdfs.append(gdf)
            print(f"✅ {category}: {len(gdf)} puntos")
        except Exception as e:
            print(f"⚠️ Error cargando {category}: {e}")
    
    # Unimos todo en una sola tabla gigante
    gdf_total = pd.concat(lista_gdfs, ignore_index=True)
    return gdf_total

# EJECUTAR LA CARGA
gdf_servicios = cargar_servicios_unificados(RUTA_GPKG, SERVICE_LAYERS)

# Asegurar sistema de coordenadas métrico (para poder hacer buffers en metros)
# EPSG:32719 es zona UTM 19S (Santiago), ideal para medir metros.
gdf_servicios = gdf_servicios.to_crs(epsg=32719) 

print(f"\n🎉 LISTO: Tenemos {len(gdf_servicios)} servicios totales listos para analizar.")

In [ ]:
# Celda 2
def obtener_servicios_en_radio(lat, lon, radio_metros=1000):
    """
    Dado un punto (lat, lon), cuenta cuántos servicios de cada tipo hay alrededor.
    """
    # 1. Crear un punto con las coordenadas del usuario
    punto_usuario = gpd.GeoDataFrame(
        geometry=gpd.points_from_xy([lon], [lat]), 
        crs="EPSG:4326"
    ).to_crs(gdf_servicios.crs) # Convertir a metros (mismo CRS que la base)
    
    # 2. Crear el buffer (el círculo alrededor del usuario)
    circulo = punto_usuario.buffer(radio_metros)
    
    # 3. Filtrar espacialmente (clip)
    # Esto busca qué servicios caen DENTRO del círculo
    servicios_cercanos = gdf_servicios[gdf_servicios.intersects(circulo.iloc[0])]
    
    # 4. Contar por tipo
    conteo = servicios_cercanos['tipo_servicio'].value_counts().to_dict()
    
    # Rellenar con 0 los servicios que no se encontraron
    for servicio in SERVICE_LAYERS.keys():
        if servicio not in conteo:
            conteo[servicio] = 0
            
    return conteo

In [ ]:
# ============================================================================
# CELDA 3: PRUEBA DE FUEGO (NUMÉRICA)
# ============================================================================
# Probemos con una ubicación conocida: Plaza de Armas de Santiago
lat_prueba = -33.4372
lon_prueba = -70.6506

print(f"📍 Analizando ubicación: {lat_prueba}, {lon_prueba}")
print("⏳ Escaneando servicios en 1000 metros a la redonda...")

# Ejecutamos tu función
resultado = obtener_servicios_en_radio(lat_prueba, lon_prueba, radio_metros=1000)

# Mostramos el resultado ordenado
series_resultado = pd.Series(resultado).sort_values(ascending=False)

print("\n📊 RESULTADOS DEL ESCANEO:")
print("-" * 30)
print(series_resultado)
print("-" * 30)
print(f"TOTAL SERVICIOS ENCONTRADOS: {series_resultado.sum()}")

# Cálculo simple de puntaje (Ejemplo: 1 punto por cada tipo de servicio presente)
puntaje_cobertura = (series_resultado > 0).sum()
print(f"⭐ PUNTAJE DE DIVERSIDAD: {puntaje_cobertura} / {len(SERVICE_LAYERS)} categorías cubiertas")

In [ ]:
# ============================================================================
# CELDA 4: VISUALIZACIÓN DEL "RADAR"
# ============================================================================
import matplotlib.pyplot as plt

def graficar_radar(lat, lon, radio_metros=1000):
    """
    Función visual para ver qué atrapó el buffer
    """
    # 1. Crear el punto y el buffer igual que en tu lógica
    punto = gpd.GeoDataFrame(
        geometry=gpd.points_from_xy([lon], [lat]), 
        crs="EPSG:4326"
    ).to_crs(gdf_servicios.crs)
    
    circulo = punto.buffer(radio_metros)
    
    # 2. Filtrar los servicios que caen dentro
    servicios_dentro = gdf_servicios[gdf_servicios.intersects(circulo.iloc[0])]
    
    # --- PLOT ---
    fig, ax = plt.subplots(figsize=(10, 10))
    
    # A. Dibujar el círculo del radio (El Radar)
    gpd.GeoSeries(circulo).plot(ax=ax, color='blue', alpha=0.1, edgecolor='blue', linewidth=2)
    
    # B. Dibujar los servicios encontrados (puntos de colores)
    if not servicios_dentro.empty:
        servicios_dentro.plot(ax=ax, column='tipo_servicio', cmap='tab20', markersize=25, legend=True, legend_kwds={'loc': 'upper left', 'bbox_to_anchor': (1, 1)})
    
    # C. Dibujar el punto del usuario (Tu ubicación)
    punto.plot(ax=ax, color='red', markersize=100, marker='*', label='Tu Ubicación')
    
    # Configuración del mapa
    ax.set_title(f"Radar de Servicios ({radio_metros}m)", fontsize=15)
    ax.set_xlabel("Metros Este")
    ax.set_ylabel("Metros Norte")
    ax.grid(True, linestyle="--", alpha=0.5)
    
    plt.show()

# ¡Probamos la gráfica con los mismos datos!
graficar_radar(lat_prueba, lon_prueba, radio_metros=1000)